# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# LLM Chain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [10]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

In [12]:
product = "Queen Size Sheet Set"
chain.run(product)

'Royal Linens Co.'

# SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

#prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

#Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
#prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)

#chain 2
chain_two = LLMChain(llm=llm, prompt= second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens.
Royal Linens is a luxury bedding company that offers high-quality linens and bedding accessories for a comfortable and stylish home.

> Finished chain.


'Royal Linens is a luxury bedding company that offers high-quality linens and bedding accessories for a comfortable and stylish home.'

# SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [23]:
#prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English: {Review}"
)

#chain 1: input= Review, output= English_Review
chain_one = LLMChain(
    llm=llm, 
    prompt=first_prompt, 
    output_key="English_review"
)

In [24]:
#prompt template 2: Summarize review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review into 1 sentence: {English_review}"
)

#chain 1: input= English_review, output= summary
chain_two = LLMChain(
    llm=llm, 
    prompt=second_prompt,
    output_key="summary"
)
    

In [25]:
#prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review in: {Review}"
)

#chain 1: input= Review, output= language
chain_three = LLMChain(
    llm=llm, 
    prompt=third_prompt,
    output_key="language"
)

In [26]:
#prompt template 4: follow up message
forth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language:\
    Summary: {summary}  Language: {language}"
)

#chain 4: input= suummary, language, output= followup_message
chain_four = LLMChain(
    llm=llm, 
    prompt=forth_prompt,
    output_key="followup_message"
)

In [28]:
#overall_chain: input = Review output = English_review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_review", "summary", "followup_message"],
    verbose=True
)

In [29]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer finds the taste mediocre and suspects that the product may be an old batch or counterfeit as the foam doesn't hold well.",
 'followup_message': "Le critique trouve le goût médiocre et soupçonne que le produit pourrait être un vieux lot ou une contrefaçon car la mousse ne tient pas bien. Il est possible que le produit ait été altéré ou stocké de manière inappropriée, ce qui a affecté sa qualité. Il est important de vérifier la date de péremption et l'authenticité du produit avant de l'acheter. Si le problème persiste, il est recommandé de contacter le fabricant pour obtenir des informations supplémentaires."}

# Router Chain

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [38]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [40]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template= p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [41]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [42]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [45]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [46]:
chain = MultiPromptChain(
    router_chain= router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [49]:
# Change this prompt, and notice that the route is different based on the type of prompt
chain.run("what is the square root of 48")



> Entering new MultiPromptChain chain...
math: {'input': 'what is the square root of 48'}
> Finished chain.


'The square root of 48 can be broken down into its prime factors: 48 = 2 x 2 x 2 x 2 x 3. We can then group the factors into pairs of 2: (2 x 2) x (2 x 2) x 3. Each pair can be simplified to 2, so we are left with 2 x 2 x √3. Simplifying further, we get 4√3 as the answer.'